### solution_v1.2

### 1. 环境初始化与数据加载
首先，我们导入numpy,pandas计算库，并读取原始数据集，并展示数据集基础信息

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

data_path = 'source_data'
train_data = pd.read_csv(data_path +'/train.csv')
test_data = pd.read_csv(data_path +'/test.csv')
# 查看训练数据的信息
train_data.info()
#train_data.columns

可以看到train_data共 891 个样本，其中所包含属性如下
- PassengerId => 乘客ID
- Survived => 乘客是否存活
- Pclass => 乘客等级(1/2/3等舱位)
- Name => 乘客姓名
- Sex => 性别
- Age => 年龄:缺失了 177 个值 ,   缺失率19.8%
- SibSp => 堂兄弟/妹个数
- Parch => 父母与小孩个数
- Ticket => 船票信息
- Fare => 票价
- Cabin => 客舱号码：缺失了  687 个值， 缺失率77.1%
- Embarked => 登船港口：缺失了 2 个值 缺失率0.2%

In [ ]:
test_data.info()

可以了解到test_data一共 418 个样本，其中缺失属性如下
- Age => 年龄:缺失了 86 个值 ,   缺失率20.5%
- Fare => 票价:缺失了 1 个值 ，  缺失率0.2%
- Cabin => 客舱号码：缺失了  327 个值， 缺失率78.2%

In [ ]:
train_data.describe()

### 2.探索性数据分析 ：
在建模之前，先分析可能影响的因素。通过可视化手段分析:
性别和年龄是否影响逃生机会（Sex, Age）
仓位登记和票价是否影响逃生机会？（Pclass, Fare）
登船地点是否存在隐含的生存规律？（Embarked）
下面的图表将分别从分类特征和数值分布两个维度揭示这些规律。

In [ ]:
# embarked 缺失值为2个，直接用众数填充
train_data['Embarked'] = train_data['Embarked'].fillna(train_data['Embarked'].mode()[0])

In [ ]:
# Fare 缺失值为1个，用 Train 的中位数去填，假设他买了张普通票
# 确保计算人均票价正常注意使用train_data的中位数
test_data['Fare'] = test_data['Fare'].fillna(train_data['Fare'].median())

In [ ]:
# Sex  分析男性和女性中各自的比率
survival_by_sex = train_data.groupby('Sex')['Survived']
print(survival_by_sex.mean)

# 绘图
fig, axes = plt.subplots(figsize=(4, 3)) 
survival_by_sex.sum().plot(kind='bar', color=['skyblue'])
axes.set_title('Survival Count by Sex', fontsize=14)
axes.set_ylabel('Survival Count')
axes.set_xlabel('Sex')
axes.tick_params(axis='x', rotation=0)
plt.show()

In [ ]:
# Pclass  分析不同舱位的生存率
survival_by_pclass = train_data.groupby('Pclass')['Survived'].mean()
print(survival_by_pclass)

# 绘图
plt.figure(figsize=(4, 3))
sns.countplot(train_data, x='Pclass', hue='Survived')
plt.title('Survival Count by Pclass')
plt.show()

In [ ]:
# Embarked  分析不同登船港口的生存率
survival_by_embarked = train_data.groupby('Embarked')['Survived'].mean()
print(survival_by_embarked)

# 绘图
plt.figure(figsize=(4, 3))
sns.countplot(train_data, x='Embarked', hue='Survived')
plt.title('Survival Count by Embarked')
plt.show()


### 3. 特征工程：高级特征构造 
- Title ：从 Name 提取 Mr, Mrs, Miss, Master。这极大地帮助树模型划分社会地位。
- FarePerPerson 从ticker中提取团体票和个人票，更真实的反应每个人的票价
- FamilySize ：SibSp + Parch + 1。将原始数据中的 SibSp 和 Parch 合并为 FamilySize。树模型可以很容易找到 FamilySize > 4 存活率骤降的分割点。
进一步降噪，鲁棒性，非线性表达。将familysize 分为3组 独自1人/ 2-4人/5人及以上
- IsAlone ：基于 FamilySize 创建的二分类特征 (0/1)。

In [ ]:
# 提取名字中的称谓
#[A-Za-z]表示匹配任意字母，+表示匹配前面的字符一次或多次，\.表示匹配点号本身 \转移符
train_data['Title'] = train_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
test_data['Title'] = test_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
# 查看不同称谓的分布情况
# print(train_data['Title'].value_counts())
# 将一些罕见的称谓归类为'Other'，注意要记录test的特殊数据
rare_titles = ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 
               'Rev', 'Sir', 'Jonkheer', 'Dona']
#replace函数用于替换指定的值 ,第一个参数是要被替换的值，第二个参数是替换成的值
train_data['Title'] = train_data['Title'].replace(rare_titles, 'Other')
test_data['Title'] = test_data['Title'].replace(rare_titles, 'Other')
# 将常见的称谓进行统一
train_data['Title'] = train_data['Title'].replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'})
test_data['Title'] = test_data['Title'].replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'})

# 检查是否有未映射的 Title
if train_data['Title'].isnull().any():
    print("警告：有 Title 没被映射成功！")
    print(train_data[train_data['Title'].isnull()])

if test_data['Title'].isnull().any():
    print("警告：有 Title 没被映射成功！")
    print(test_data[test_data['Title'].isnull()])

# 查看处理后的称谓分布情况
# print(train_data['Title'].value_counts())
# 分析不同称谓的生存率
survival_by_title = train_data.groupby('Title')['Survived'].mean()  
print("不同称谓的生存率：")
print(survival_by_title)
# 查看现在的 Title 各个值的数量
print(train_data['Title'].value_counts())

In [ ]:
# 基于ticket号，计算人均票价
all_data = pd.concat([train_data, test_data], sort=False)
ticket_counts = all_data.groupby('Ticket').size()
print(ticket_counts)
# 映射原数据
train_data['TicketCount'] = train_data['Ticket'].map(ticket_counts)
test_data['TicketCount'] = test_data['Ticket'].map(ticket_counts)   
# 计算人均票价
train_data['FarePerPerson'] = train_data['Fare'] / train_data['TicketCount']
test_data['FarePerPerson'] = test_data['Fare'] / test_data['TicketCount']   

In [ ]:
# 构造家庭大小为新特征
train_data["FamilySize"] = train_data["SibSp"] + train_data["Parch"] + 1
test_data["FamilySize"] = test_data["SibSp"] + test_data["Parch"] + 1

# 根据家庭规模创建家庭组别特征 分箱
train_data['FamilyGroup'] = 0
# 创建家庭规模类别特征   分别为 单身(0) 小家庭(1) 大家庭(2)
train_data.loc[(train_data['FamilySize'] >= 2) & (train_data['FamilySize'] <= 4), 'FamilyGroup'] = 1
train_data.loc[train_data['FamilySize'] >= 5, 'FamilyGroup'] = 2


# test进行同样的处理
test_data['FamilyGroup'] = 0
test_data.loc[(test_data['FamilySize'] >= 2) & (test_data['FamilySize'] <= 4), 'FamilyGroup'] = 1
test_data.loc[test_data['FamilySize'] >= 5, 'FamilyGroup'] = 2
print("\n分组后的生存率统计：")
print(train_data[['FamilyGroup', 'Survived']].groupby('FamilyGroup').mean())
print("创建完成")

### 4.高级数据清洗：基于随机森林的 MICE 填补
数据属性中较少的缺失值采用简单的均值/中位数/众数填充
数据属性中缺失值较多的采用 Iterative Imputer技术。 简单来说，将“预测年龄”视为一个回归问题：利用 Pclass, Fare, FamilySize 等已知信息，通过随机森林回归模型来精准反推缺失的 Age。

In [ ]:
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor


#缺失值填补用FamilySize 模型训练用FamilyGroup
# 生存率呈现 "中间高，两头低" 的非线性关系，使用FamilyGroup分组后模型更容易学
features_for_impute = ['Pclass', 'Sex', 'FamilySize', 'Fare', 'Age', 'Embarked', 'Title', 'FarePerPerson']
df_train_impute = train_data[features_for_impute].copy()


# 2.独热编码：把所有特征数字化
# 'Sex' -> 0 / 1
df_train_impute['Sex'] = df_train_impute['Sex'].map({'male': 0, 'female': 1})
# 'Embarked' -> 0 / 1 / 2
df_train_impute['Embarked'] = df_train_impute['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}) 
# 'Title' -> 0 / 1 / 2 / 3 / 4
df_train_impute['Title'] = df_train_impute['Title'].map({'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3, 'Other': 4})


# 3. 配置填充器：IterativeImputer
# estimator: 采用“随机森林回归”来预测缺失值
rf_imputer = IterativeImputer(
    estimator=RandomForestRegressor(
        n_jobs=-1,              # n_jobs=-1 使用所有可用的 CPU 核心来加速训练
        random_state=42
    ),
    max_iter=10,                # max_iter: 迭代轮数10次
    random_state=42             # 设置随机种子，使得每次运行结果相同
)

# 4. 训练阶段 (Fit) - 此时只看不做
# 这一步，rf_imputer 只是在“观察”数据，寻找年龄和其他特征的规律
print("正在学习训练集的规律...")
rf_imputer.fit(df_train_impute) 

# 5. 应用阶段 (Transform) - 此时只做不想
# 这一步，用刚才学到的规律，把训练集自己的坑填上
print("正在填充训练集...")
imputed_data = rf_imputer.transform(df_train_impute)



# 6. 还原回 DataFrame
df_filled = pd.DataFrame(imputed_data, columns=features_for_impute)

# 把填好的 Age 塞回原始数据
train_data['Age'] = df_filled['Age'].round(0)

# 7. 修正非法值
# 如果预测出负数，强制变为 1；如果太大，限制在 80
train_data.loc[train_data['Age'] < 0, 'Age'] = 1
train_data.loc[train_data['Age'] > 80, 'Age'] = 80

print("基于随机森林的智能填充完成！")
print(train_data[['Age', 'Embarked']].isnull().sum()) # 检查缺失值数量
print("Age 缺失值填充完毕!前10个结果预览:")
print(train_data['Age'].head(10))
print("填补后的 Master 平均年龄:", train_data[train_data['Title'] == 'Master']['Age'].mean())

### 5.测试集填补 

In [ ]:
# 修复 Test 集的 缺失值- 
# Age => 年龄:缺失了 86 个值 ,   缺失率20.5%
# Fare => 票价:缺失了 1 个值 ，  缺失率0.2%


# 2. 修复 Age (86 个缺失值)
# 用训练好的 rf_imputer 去预测 Test 的年龄
# 第一步：准备数据 (格式必须和训练 imputer 时一模一样)
features_for_impute = ['Pclass', 'Sex', 'FamilySize', 'Fare', 'Age', 'Embarked','Title', 'FarePerPerson']
df_test_impute = test_data[features_for_impute].copy()

# 3.独热编码 - 特征数字化 (Sex/Embarked 转数字)
df_test_impute['Sex'] = df_test_impute['Sex'].map({'male': 0, 'female': 1})
df_test_impute['Embarked'] = df_test_impute['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
df_test_impute['Title'] = df_test_impute['Title'].map({'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3, 'Other': 4})  

# 4.应用 Train训练集 学到的规律 (Transform)，只需要 transform
test_imputed_data = rf_imputer.transform(df_test_impute)

# 5.把填好的 Age 填充 test_data
df_test_filled = pd.DataFrame(test_imputed_data, columns=features_for_impute)
test_data['Age'] = df_test_filled['Age'].round(0).astype(int)

# 6.修正非法值
test_data.loc[test_data['Age'] < 0, 'Age'] = 1
test_data.loc[test_data['Age'] > 80, 'Age'] = 80


print("现在剩余的 NaN 情况：")
print(test_data[['Age', 'Fare']].isnull().sum()) # 查看缺失值数量



### 6.模型训练与提交
我们使用 随机森林分类器

In [ ]:
y = train_data["Survived"]

features = ["Pclass", "Sex", "Age", "Embarked", "FamilyGroup", "FarePerPerson", 'Title']
# 独热编码
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

# 自动补齐测试集中缺失的列（补为0），并确保顺序一致
X_test = X_test.reindex(columns=X.columns, fill_value=0)
print(f"训练集形状: {X.shape}")
print(f"测试集形状: {X_test.shape}")

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
# 网格搜索交叉验证调参

# 定义“尝试”的参数范围 (Param Grid)
# 模型训练与预测 
'''

param_grid = {
    'n_estimators': [100, 400, 700],        # 树越多越稳定
    'max_depth': [4, 6, 8, 10],                 # 树越深学得越细，但也越容易过拟合
    'min_samples_leaf': [2, 4, 6],              # 叶子节点最少样本数：太小会过拟合
    'min_samples_split': [2, 4, 10],        # 分裂门槛：太小会过拟合
}

model = RandomForestClassifier(
        n_jobs=-1,              # 使用所有可用的 CPU 核心来加速训练
        random_state=42         # 设置随机种子，使得每次运行结果相同
)

# 4. 创建“搜索器” (GridSearchCV)
# cv=5: 意思是做 5折交叉验证 (把数据切成5份，轮流验证，防止偶然性)
# n_jobs=-1: 调用你电脑所有的 CPU 核心一起跑，加速
# verbose=2: 打印进度条，数字1表示打印少量信息，数字越大信息越多
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           cv=5, verbose=2)

grid_search.fit(X, y)


print(f" 最佳参数组合: {grid_search.best_params_}")
print(f" 最佳验证集分数: {grid_search.best_score_:.4f}")
print("-" * 30)


best_rf = grid_search.best_estimator_

最佳参数组合: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 400}
最佳验证集分数: 0.8451
'''
model = RandomForestClassifier(
        n_estimators=400,        # 树越多越稳定
        max_depth=10,                 # 树越深学得越细，但也越
        min_samples_leaf=2,              # 叶子节点最少样本数：太小会过拟合
        min_samples_split=2,        # 分裂门槛：太小会过
        n_jobs=-1,              # 使用所有可用的 CPU 核心来加速训练
        random_state=42         # 设置随机种子，使得每次运行结果相同
)

model.fit(X, y)
predictions = model.predict(X_test)


output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

### 7.模型评估：交叉验证 (CV)
采用交叉验证 (Cross Validation) 分析模型的表现。通过将训练数据切分为 5 份，轮流进行“模拟考试”，得到一个比单一分数更可信的平均分和标准差，从而判断模型的泛化能力。

In [ ]:
from sklearn.model_selection import cross_val_score
# cv=5 表示考 5 次，每次考题都不一样
scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print(f"每次考试的分数: {scores}")
print(f"本地估算平均分: {scores.mean():.4f}")
print(f"标准差 (越小越稳): {scores.std():.4f}")